In [2]:
import numpy as np

In [9]:
print(np.array([[1,2,3],[4,5,6]]))

[[1 2 3]
 [4 5 6]]


In [25]:
import scipy as sp

In [26]:
num=np.random.randint(1,20,size=(1,15))[0]
print(num)

[13  8  8  5 18  7 16 14 10 18 15 16 19  8 17]


In [27]:
sp.mean(num)

12.8

In [29]:
from scipy import stats
sp.stats.skew(num)

-0.2516050116651928

In [38]:
import matplotlib.pyplot as plt
from nltk.parse.stanford import StanfordParser


In [92]:
#NLP using corenlp model
sentence="This topic is easier than other topic"
scp=StanfordParser(path_to_jar="D:/study/ml/stanford/stanford-corenlp-full-2020-04-20/stanford-corenlp-4.0.0-javadoc.jar",path_to_models_jar='D:\study\ml\stanford\stanford-corenlp-full-2020-04-20\stanford-corenlp-4.0.0-models.jar')

scp

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
#parse tree
#result=list(scp.raw_parse(sentence))

# student recommendation system

In [43]:

import pandas as pd

#import data
data=pd.read_csv("students_record.csv")
data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


In [45]:
feature_name=['OverallGrade','Obedient','ResearchScore','ProjectScore']
training_feature=data[feature_name]

Outcome=['Recommend']
Outcome_label=data[Outcome]

#view Features
training_feature

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


In [54]:
#divide feature based on type
num=['ResearchScore','ProjectScore']
categorical=['OverallGrade','Obedient']

In [55]:
#numeric feature scaling
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

ss.fit(training_feature[num])
training_feature[num]=ss.transform(training_feature[num])
training_feature

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Programs\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


In [56]:
#categorical to 0-1
training_feature=pd.get_dummies(training_feature,columns=categorical)
training_feature

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,1,0,0,0,0,0,1
1,0.730648,-0.091777,0,0,1,0,0,1,0
2,-1.803390,-1.560203,0,0,0,0,1,1,0
3,0.392776,0.772004,0,1,0,0,0,0,1
4,-1.465519,-0.998746,0,0,0,1,0,1,0
5,0.967158,1.117516,1,0,0,0,0,0,1
6,-0.114032,0.253735,0,1,0,0,0,0,1
7,0.392776,-0.869179,0,0,1,0,0,0,1


In [59]:
# get list of new categorical features
categorical_engineered_features = list(set(training_feature.columns) - set(num))
categorical_engineered_features

['OverallGrade_E',
 'OverallGrade_B',
 'OverallGrade_F',
 'Obedient_N',
 'Obedient_Y',
 'OverallGrade_C',
 'OverallGrade_A']

# Classification Modeling

In [61]:
from sklearn.linear_model import LogisticRegression

#fit model
lr=LogisticRegression()
model=lr.fit(training_feature,np.array(Outcome_label['Recommend']))

model

D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Evaluation

In [63]:
# simple evaluation on training data
pred_labels=model.predict(training_feature)
actual_labels=np.array(Outcome_label['Recommend'])

# evaluate model performance
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('Accuracy:',(accuracy_score(actual_labels,pred_labels)*100))
print('Classification stats:',(classification_report(actual_labels,pred_labels)))

Accuracy: 100.0
Classification stats:               precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



# Deployment

In [2]:
from sklearn.externals import joblib

if not os.path.exists('Model'):
    os.mkdir('Model')
if not os.path.exists('Scalar'):
    os.mkdir('Scalar')
    
joblib.dump(model,r'Model/model.pickle')
joblib.dump(ss,r'Scalar/scalar.pickle')

D:\Programs\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


NameError: name 'os' is not defined

# Prediction

In [ ]:
#load
model=joblib.load('Model/model.pickle')
scaler=joblib.load('Scalar/scalar.pickle')

In [71]:
#predict custom data
inp=pd.DataFrame([{'Name':'Hasan','OverallGrade':'A','Obedient':'N','ResearchScore':85,'ProjectScore':70
                  
                    },{'Name':'Haider','OverallGrade':'F','Obedient':'Y','ResearchScore':45,'ProjectScore':30
                      }])
inp=inp[['Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']]
inp

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore
0,Hasan,A,N,85,70
1,Haider,F,Y,45,30


In [77]:
#data preparation
pred_feature=inp[feature_name]
#scaling numerical
pred_feature[num]=scaler.transform(pred_feature[num])
#categorical
pred_feature=pd.get_dummies(pred_feature,columns=categorical)
pred_feature

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Programs\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y
0,85.0,70.0,1,0,1,0
1,45.0,30.0,0,1,0,1


In [79]:
#add missing cols
categorical_present=set(pred_feature)-set(num)
missing=set(categorical_engineered_features)-categorical_present
for feature in missing:
    pred_feature[feature]=[0]*len(pred_feature)
pred_feature

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y,OverallGrade_E,OverallGrade_C,OverallGrade_B
0,85.0,70.0,1,0,1,0,0,0,0
1,45.0,30.0,0,1,0,1,0,0,0


In [81]:
#predict
pred=model.predict(pred_feature)
pred
inp['Recommend']=pred
inp

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Hasan,A,N,85,70,Yes
1,Haider,F,Y,45,30,Yes


In [84]:
## data retrieval
new_data = pd.DataFrame([{'Name': 'Nathan', 'OverallGrade': 'F', 'Obedient': 'N', 'ResearchScore': 30, 'ProjectScore': 20},
                  {'Name': 'Thomas', 'OverallGrade': 'A', 'Obedient': 'Y', 'ResearchScore': 78, 'ProjectScore': 80}])
new_data = new_data[['Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']]
new_data
prediction_features = new_data[feature_name]

# scaling
prediction_features[num] = scaler.transform(prediction_features[num])

# engineering categorical variables
prediction_features = pd.get_dummies(prediction_features, columns=categorical)

# view feature set
prediction_features

# add missing categorical feature columns
current_categorical_engineered_features = set(prediction_features.columns) - set(num)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features
for feature in missing_features:
    # add zeros since feature is absent in these data samples
    prediction_features[feature] = [0] * len(prediction_features) 

# view final feature set
prediction_features

## predict using model
predictions = model.predict(prediction_features)

## display results
new_data['Recommend'] = predictions
new_data

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\Programs\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Nathan,F,N,30,20,Yes
1,Thomas,A,Y,78,80,Yes


In [ ]:
#Not accurate
#not able to scale numerical data